# Exploratory Activity 10: Principal Component Analysis of Radiative Forcing Data

*adapted from [GEOG 595: Geographic Data Analysis](https://pjbartlein.github.io/GeogDataAnalysis/lec16.html) by Prof. Patrick Bartein*

Today we will use principal component analysis to examine annual changes in the radiative forcing associated with 11 environmental variables since the year 1750, as calculated in the Intergovernmental Panel on Climate Change’s [Fifth Assessment Report](http://www.ipcc-data.org/observ/index.html):

<ul>
  <li>solar activity</li>
  <li>stratospheric ozone</li>
  <li>stratospheric water vapor</li>
  <li>volcanic activity</li>
  <li>contrails</li>
  <li>tropospheric ozone</li>
 <li>total aerosol</li>
 <li>black carbon deposition on snow</li>
 <li>land use changes</li>
 <li>carbon dioxide</li>
 <li>other greenhouse gases</li>
</ul>


__10.1 Calculating Eigenvectors__

**Question 10.1.1** As you have learned, Earth's climate system contains many interdependencies and feedbacks, and thus it is useful to ascertain which environmental factors are closely correlated and which assert their own distinct forcing on the Earth's energy balance. Based on your understanding of the environmental factors provided, make a qualitative prediction about which you expect to be closely correlated and which you expect to behave in their own distinct manner.


Now we will consult the IPCC data to make a quantitative assessment of the qualitative prediction you made above. Use the code below to load and trim the radiative forcing data.


In [ ]:
# import some useful libraries
from scipy.io import loadmat
import numpy as np

# load the data
data = loadmat('data/RadForcing.mat')
X = data['RadForcing']

# remove first column, which contains unnecessary year labels
X = X[:,1:]

# display the number of rows and columns in the data variable X
X.shape

You will see that the data matrix "X" contains 11 variables (as expected), with 262 observations each (as the IPCC report contains calculations through 2011). For the purposes of interpretability that will become clear later on, use the code below to invert the signs of two of these variables.

In [ ]:
# invert the signs of the fourth and eleventh variables
X[:,3] = -X[:,3]
X[:,10] = -X[:,10]

**Question 10.1.2** Next, normalize the data into "standard units."

In [ ]:
# create a variable the same size as the "X" data matrix
X_std = X

# iterate over all of the columns in "X"
for i in np.arange(X.shape[1]):
    
    # for each data column, substract off the mean and divide by the standard deviation
    X_std[:,i] = (X[:,i] - np.mean(X[:,i])) / np.std(X[:,i])
    
##NOTE THAT THIS METHOD ALSO CHANGES THE ORIGNIAL MATRIX X. POINT THIS OUT TO STUDENTS IF THEY USE THIS METHOD. 

**Question 10.1.3** Next, compute the covariance matrix of the normalized data matrix, defined as 𝜮 below. Here, m is the number of observations in each column and the superscript T indicates the transpose of a matrix.


<img src="https://latex.codecogs.com/svg.latex?\Large&space;\sum=\frac{1}{m}X^TX" title="" />

*Hint: You may find the functions np.transpose and np.dot useful!*

In [ ]:
# calculate the covariance matrix
X_cov = (1 / X_std.shape[0]) * np.dot(np.transpose(X_std),X_std)

Finally, use the code below to run a built-in numpy function that will obtain the eigenvectors and eigenvalues of the covariance matrix:

In [ ]:
# see https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html
eigen_vals, eigen_vecs = np.linalg.eig(X_cov)

The output variable "eigen_vecs" above gives the "principal components" of the normalized data matrix, the axes or directions which explain the most variance in the original data set.

## 10.2 Dimensionality Reduction

While 11 eigenvectors were calculated, not all of them are equally important in explaining the variance in the data. Thus, by selecting a subset of the eigenvectors, we can reduce the dimensionality of the original data set. This raises the question: how many eigenvectors should you preserve?


**Question 10.2.1** The magnitude of the corresponding eigenvalues gives a measure of the importance of each eigenvector. Calculate the percent of the total variance explained by each eigenvector by dividing each individual eigenvalue by the sum of all the eigenvalues.


In [ ]:
pct_var_explained = eigen_vals/sum(eigen_vals)

**Question 10.2.2** Use the percent of the total variance explained by each eigenvector to make a plot of the "cumulative variance" explained by the first *n* eigenvectors (i.e., the percent of the total variance explained by each eigenvector plus the percentages explained by the eigenvectors before it). 

*Note: Python does not necessarily report the eigenvectors in order of importance, so it would be wise to sort the percent variance variable from highest to lowest before making the cumulative variance plot.*

In [ ]:
# sort the percent variance variable in ascending order
# then flip to obtain the desired highest-to-lowest ranking
pct_var_explained = np.flip(np.sort(pct_var_explained))

# create a list for storing cumulative variances
# use the first, largest percent variance explained as its first entry
cumul_var_explained = [pct_var_explained[0]]

# iterate over the remainder of ther percent variance variable (starting with the second entry)
for pct_var in pct_var_explained[1:]:
    
    # obtain the cumulative variance explained by all of the previous eigenvectors
    var_explained_last_time = cumul_var_explained[-1]
    
    # add to that the percent variance explained the next eigenvector
    total_var_explained_now = var_explained_last_time + pct_var
    
    # append the new cumulative variance explained to the cumul_var_explained variable
    cumul_var_explained.append(total_var_explained_now)
    
# create a subplot instance and store referenes to the figure and axis
import matplotlib.pyplot as plt
fig, ax = plt.subplots(num=None, facecolor='w', edgecolor='k')

# add plots to the axis using the number of eigenvectors as the 'x'
# and the cumulative variance explained by the first x eigenvectors for the 'y'
ax.plot(np.arange(len(cumul_var_explained)),cumul_var_explained,linewidth=.5)

# add labels to the axes
plt.ylabel('Cumul. Var. Explained')

**Question 10.2.3** The choice of how many eigenvectors to preserve depends on the data set and your particular goals, but many data scientists choose to preserve however many principal components are necessary to explain 95-99% of the total variance. Using this criteria, how many eigenvectors should you preserve in your case?


**Question 10.2.4** Examine the principal components you've selected by printing the first n columns of your "eigen_vecs" variable below.


In [ ]:
print(eigen_vecs[:,0:4])

## 10.3 Analyzing the Loadings of the Principal Components

By themselves, these principal components are not particularly informative. It would be useful to know how they relate to each of the environmental variables in the original data set.

To obtain this information, we "load" the magnitude of the eigenvalues onto direction indicated by the unit eigenvectors (all with lengths of 1). The resultant "loadings," calculated by the code below, describe the covariances between the original variables and the principal components.


In [ ]:
loadings = eigen_vecs * np.sqrt(eigen_vals)

**Question 10.3.1** Print the first column of the "loadings" variable to display the covariance of the first principal component with each of the 11 environmental variables.


In [ ]:
print(loadings[:,0])

**Question 10.3.2** Which environmental variables appear to be contained in this first principal component (i.e., which exhibit high covariances with the first eigenvector)? Which environmental variables seem unrelated to this principal component? Note that the environmental variables in the original data matrix you imported are not in any particular order relative to the list provided at the beginning of this activity; just refer to them by their column indices for now.


*Hint: It is the absolute value of the loading that indicates covariance; sign is unimportant.*


**Question 10.3.3** Also examine the loadings of the other principal components you decided to preserve. Which environmental variables exhibit high covariances with these eigenvectors?


In [ ]:
print(loadings[:,1])

print(loadings[:,2])

print(loadings[:,3])

**Question 10.3.4** Verify the conclusions of your loading analysis above by making a time series of the radiative forcing attributed to each environmental factor during the 1750-2011 time period.


In [ ]:
# create a subplot instance and store referenes to the figure and axis
fig, ax = plt.subplots(num=None, facecolor='w', edgecolor='k')

# add plots to the axis using the years 1750-2011 as 'x' and each column of radiative forcings for the 'y'
ax.plot(np.arange(1750,2012),X[:,0],linewidth=.5,label='col0')
ax.plot(np.arange(1750,2012),X[:,1],linewidth=.5,label='col1')
ax.plot(np.arange(1750,2012),X[:,2],linewidth=.5,label='col2')
ax.plot(np.arange(1750,2012),X[:,3],linewidth=.5,label='col3')
ax.plot(np.arange(1750,2012),X[:,4],linewidth=.5,label='col4')
ax.plot(np.arange(1750,2012),X[:,5],linewidth=.5,label='col5')
ax.plot(np.arange(1750,2012),X[:,6],linewidth=.5,label='col6')
ax.plot(np.arange(1750,2012),X[:,7],linewidth=.5,label='col7')
ax.plot(np.arange(1750,2012),X[:,8],linewidth=.5,label='col8')
ax.plot(np.arange(1750,2012),X[:,9],linewidth=.5,label='col9')
ax.plot(np.arange(1750,2012),X[:,10],linewidth=.5,label='col10')

# add labels and title
plt.xlabel('Year')
plt.ylabel('Radiative Forcing (W/m2)')
plt.title('Radiative Forcing since 1750')

# add a legend with adjusted position
fig.legend(bbox_to_anchor=(1, 0.9),bbox_transform=plt.gcf().transFigure)

**Question 10.3.5** Do the environmental factors that exhibited high covariances with the first principal component appear somewhat correlated over time? Do the environmental factors that exhibited low covariances with the first principal component appear uncorrelated with the other variables? Be sure to mention if there are any major discrepancies between your loadings-based conclusions and your visual analysis that cannot be easily explained.


**Discussion Questions.** Pair up with a partner and discuss the following.

1. The environmental variable names in the actual order in which they appear in the RadForcing.csv data matrix (column 0 to column 10) are: carbon dioxide, other greenhouse gases, tropospheric ozone, stratospheric ozone, total aerosol, land use changes, stratospheric water vapor, black carbon deposition on snow, contrails, volcanic activity, and solar activity.
    * Given this information, which environmental factors (by name) exhibited the highest covariance with your first principal component? Did you partner come to the same conclusion? If not, try to identify the difference(s) in your approaches that may have led to this discrepancy.
    * Do the variables associated quantitatively with your first principal component align with those you qualitatively predicted to be well correlated with one another at the beginning of this exercise? If not, explain.
    * As you know, correlation does not necessarily imply causation. Are there any environmental factors that the principal component analysis identifies as correlated that you suspect indicate temporal coincidences rather than causal relationships?
2. Make a new normalized data matrix containing only the environmental variables you expected to be well correlated in your initial qualitative prediction. Calculate the covariance matrix of this new data matrix, and then compute the eigenvalues and eigenvectors for this new covariance matrix.
    * Based on the cumulative variance explained by each subsequent principal component, how many eigenvectors would you choose to preserve in this case?
    * Is the number of principal components preserved larger or smaller than when you analyzed the entire data set? Does your answer make sense, given the criteria by which you excerpted your data matrix? 
    * What answer would you expect to get if you truncated the normalized data matrix in a different way?
3. This activity represents just one of many possible applications of principal component analysis to atmospheric science, and not even the most common one! If there is time, peruse the publications below to learn about other examples of principal component analyses in action in geophysical data exploration.
    * [Identifying Surface Climate Variations](https://journals.ametsoc.org/doi/pdf/10.1175/1520-0442%281993%29006%3C1743%3ASCVOTN%3E2.0.CO%3B2)
    * [Characterizing CO2 Interannual Variability](https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/2001GL014419)
    * [Evaluating Instrument Performance](https://journals.ametsoc.org/doi/pdf/10.1175/JTECH-D-15-0085.1)